# Contents


1. Adding data set
2. About train data 
    1. Data cleaning
    2. Data exploring
        1. Basic data exploration
        2. Advance data exploration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.style as style
style.use('fivethirtyeight')
import seaborn as sns
import datetime

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Adding data set
* riiid-test-answer-prediction
* riiid-parquet-files

>Since the train dataset is huge(5G), i added the riiid_parquet_files data [here](https://www.kaggle.com/ryati131457/riiid-parquet-files) in this kernel, and intent to use the train.parquet file to load the train dataset, and it deducted the loading time from 10 minutes to about 10 second. Thanks Ryati!

In [ ]:
%%time
train_original = pd.read_parquet('../input/riiid-parquet-files/train.parquet')

print('Trian size:', train_original.shape)

In [ ]:
# import datatable as dt

# # reading data from csv using datatable and converting to pandas
# train_data = dt.fread("../input/riiid-test-answer-prediction/train.csv").to_pandas()

# # writing dataset as pickle
# train_data.to_pickle("riiid_train.pkl.gzip")

# # load pickled train data
# train_data = pd.read_pickle("../input/riiid_train.pkl.gzip")

# print("Train size:", data.shape)

# 2. About train data

* row_id: (int64) ID code for the row.

* timestamp: (int64) the time in milliseconds between this user interaction and the first event completion from that user.

* user_id: (int32) ID code for the user.

* content_id: (int16) ID code for the user interaction

* content_type_id: (int8) 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture.

* task_container_id: (int16) Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id.

* user_answer: (int8) the user's answer to the question, if any. Read -1 as null, for lectures.

* answered_correctly: (int8) if the user responded correctly. Read -1 as null, for lectures.

* prior_question_elapsed_time: (float32) The average time in milliseconds it took a user to answer each question in the previous question bundle, ignoring any lectures in between. Is null for a user's first question bundle or lecture. Note that the time is the average time a user took to solve each question in the previous bundle.

* prior_question_had_explanation: (bool) Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback.

**Problems from exploring the train data file:**

* data types are not approporate.
* having missing values.

In [ ]:
print('original data information')
train_original.info()

## 2.1 Data cleaning

### 2.1.1 Convert data types 
Accroding to the description of the train data from the riiid education, we need to convert the dtypes to the corresponding data types. Since the parquet dataset has already converted most of the datatypes, here I only need to convert the content_type_id datatype from bool to int8.

In [ ]:
# copy the original trian data for convienct exploration
train = train_original.copy()

In [ ]:
# covert content_type_id data type
train['content_type_id'] = train.content_type_id.astype('int8')

In [ ]:
train.info()

### 2.1.2 Deal with missing data
Only below two columns have missing values
* NaN values in prior_question_elapsed_time
* NAN values in prior_question_had_explanation

For here, we will keep all the missing data so thatwe can easily explain the meaning of each column in the next simple EDA section.

Later, we will drop all the nan values in the advanced EDA visualization section, since the first question in each bundle is a diagnostic question, and we focus on the question rather than the lecture.

In [ ]:
# check all nan values
train.isna().sum()

In [ ]:
# count the num of values for lectures 
train[train['content_type_id'] == 1].count()

In [ ]:
# count the num of values for lecture and first question bundles 
train[(train['prior_question_had_explanation'] != 1) & (train['prior_question_had_explanation'] != 0)].count()

### **Findings:**

All 392506 Null values in prior_question_had_explanation are for user's first question bundles, which are all onboarding diagnostic tests where they did not get any feedbacks.

One part of null values(392506) in prior_question_elapsed_time are also for a user's first question bundle, and the other part of null values(2351538 - 392506 = 1959032) are for lectures.

For here, we will keep all the missing data so thatwe can easily explain the meaning of each column in the next simple EDA section.

Later, we will drop all the nan values in the advanced EDA visualization section, since the first question in each bundle is a diagnostic question, and we focus on the question rather than the lecture.

## 2.2 Data exploring

### 2.2.1 Basic data exploration
* timestamp: the time(day) between this user interaction and the first event completion from that user
* user_id: find the number of unique users.
* content_id: find the total number of different content ids and the percentage of questions in the total content ids.
* content_type_id: find number of question and lecture in the train data respectively. (0 represents a question, 1 represents a 1ecture)
* task_container_id: find the number of unique id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id.
* user_answer: find user's answer to the question, -1 for lectures.
* answered_correctly: find the correct responses, -1 for lectures.

timestamp: ms to minute

In [ ]:
# out of memory
# 1 day = 86400000 ms
# time_spent_dis = train.timestamp.apply(lambda x: x/86400000)
# fig = plt.figure(figsize=(6,4))
# time_spent_dis.plot.hist(bins=50)
# plt.axvline(time_spent_dis.median(), color = 'r',linestyle = '--', linewidth=1)
# plt.title('Histgram of Timestamp')
# plt.xlabel('Days between this user interaction and the first event completion from that user')
# plt.show()

In [ ]:
print('After calculation, we can see that most user interactions were not active very long on the APP. \
The median interactive time is about {:,} ms (about 31 days), the mean interactive timeis about {} ms (89 days),\
the longest interactive time is about {:,} ms(1012 days). \
Since the median is small than mean, we can say that it is a right skewed distribution with some outliers.'
      .format(round(train.timestamp.median()),
              round(train.timestamp.median()), 
              round(train.timestamp.max())))


user_id:

In [ ]:
# user_id
print('Number of unique users in train dataset: {}'.format(train.user_id.nunique()))


content_id:

In [ ]:
# content_id
top_10_content_list = list(train.content_id.value_counts().sort_values(ascending=False)[:10])
print('There are {} unique content in the train set. The toal 10 most frequent used content ids are {}.'.format(
train.content_id.nunique(), top_10_content_list))

content_type_id

In [ ]:
# content_type_id
question, lecture = train.content_type_id.value_counts()
print('There are {} questions and {} lectures in the trian dataset, and questions account for {}% of the total content.'.format(
question, lecture, round(question/(question + lecture)*100,1)))

task_container_id: unique batches num

In [ ]:
# task_container_id: unique batches num
print('the number of unique batches for questions or lectures: {}'.format(
train.task_container_id.nunique()))


user_answer:

In [ ]:
# user_answer
print('0-3 are the answers to questions, -1 is no-answer (lecture).')
train.user_answer.value_counts()

answered_correctly: the correct response to questions

In [ ]:
# answered_correctly: the correct response to questions
correct_question, notcorrect_question, noanswer_lecture = train.answered_correctly.value_counts()
print('There are total {} answered questions in the train data. \
 {} questions were answered correctly and {} were not answered correctly.\
 The correct answered rate is about {}%.'.format(
    correct_question + notcorrect_question, correct_question,
    notcorrect_question, round(correct_question/(correct_question + notcorrect_question)*100,1)))

In [ ]:
# -1 for lecture, so only focus on question
answer_correct = train[train["answered_correctly"] != -1].answered_correctly.value_counts(ascending= True)
answer_correct.plot.bar(tick_label = ['correct', 'not correct'])
for i,v in zip(answer_correct.index, answer_correct.values):
    plt.text(i,v,'{:,}'.format(v))
    
plt.xticks([0,1],labels = ['not correct', 'correct'], rotation= 360)
plt.show()

### 2.2.2 Advanced data exploration

**Exploring possible features**
1. total time(ms) spent on the APP
2. Total number of questions answered in the Riiid APP
3. the average rate of questions answered correctly for each user
4. the average rate of whether a user saw an explanation after the prior question

get all users' question data, and explore 4 possible features:

In [ ]:
# drop all lectures and the first question of each bundle
train = train.dropna()

In [ ]:
# get all users' question content type data 
users = train.groupby('user_id')

# get correct questions
user_answers = users['answered_correctly'].agg(correctly_answered_mean = 'mean', 
                correctly_answered = 'sum', total_answered = 'count')
user_answers['correctly_answered'] = user_answers.correctly_answered.astype('int64')

# get the total spent time(ms)
user_time = users['timestamp'].agg(total_time_spent = 'max')

# concate two dataframe
user_correct = pd.concat([user_answers,user_time], axis = 1)

# get the average of prior_question_had_explanation
user_had_explanation = users['prior_question_had_explanation'].\
    agg(had_explanation_mean = 'mean')

#concatenate user_answers and user_time two data frame to one
user_correct_info = pd.concat([user_correct, user_had_explanation], axis = 1)
user_correct_info.reset_index(inplace=True)

In [ ]:
# user_correct_info.head(10)

### EDA visualization 1:

total_time_spent

1. Total time spent on the App

In [ ]:
# change ms to day, out of memory
# 1 day = 86400000 ms
#user_correct_info['total_time_spent'] = user_correct_info.total_time_spent.apply(lambda x: x/86400000)


In [ ]:
# distribution of Timestamps (total time spent on App)
fig = plt.figure(figsize=(8,4))
user_correct_info["total_time_spent"].plot.hist(bins=50)

plt.axvline(user_correct_info["total_time_spent"].median(), color = 'r',linestyle = '--', linewidth=1)
plt.axvline(user_correct_info["total_time_spent"].mean(), color = 'y',linestyle = '--', linewidth=1)
plt.title('Distribution of Total Time (ms) Spent on App')
plt.xlabel('Total time (ms)')
plt.show()

In [ ]:
col = user_correct_info["total_time_spent"]
print('From the distribution above, we can see that most user interactions were not active very long on the APP. \
The median, mean, and longest interactive time between a user interaction and the first event completion from that user \
is about {}, {}, and {:,} days, respectively. \
From the graph above we can see it is a right skewed distribution with some outliers.'.format(
round(col.median()/86400000), round(col.mean()/86400000), round(col.max()/86400000)))

### EDA visualization 2:

prior_question_had_explanation
1. Number of answers with a prior question had explanation
2. Number of answers without a prior question had explanation


In [ ]:
# had explanation
answers_with_explanation = train[train["prior_question_had_explanation"] == 1]\
                            .answered_correctly.value_counts(ascending=True)
title1 = 'Answers with prior question had explanation'

# no explanation
answer_without_explanation = train[train["prior_question_had_explanation"] == 0]\
                        .answered_correctly.value_counts(ascending=True)
title2 = 'Answers without prior question had explanation'

def plotbarh(data, title, ylabel='Answers', xlabel='Counts'):
    plt.figure(figsize=(6,4)) 
    data.plot.barh()
    
    for i,v in zip(data.values, data.index):
        plt.text(i,v,'{:,}'.format(i))
    plt.yticks([0,1],labels = ['not correct', 'correct'], rotation= 360)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.show()

In [ ]:
plotbarh(answers_with_explanation, title1)

In [ ]:
plotbarh(answer_without_explanation, title2)

### **Findings:**

If a user saw an explanation and the correct response(s) after answering the previous question, the number of correct answers is more than twice the number of incorrect answers (60 million, 29 million).

If the user did not see the explanation and the correct answer after answering the previous question, there is no significant difference between the number of correct answers and the number of incorrect answers (4.6 million, 4.5 million).